3/28-6/10/2023 Melody

Gesture dictionary: https://docs.google.com/spreadsheets/d/1sbqk3yhHmSTHwnPzFaysUrvq9zIcPBIviqJx4oQVA1s/edit#gid=0

## 0. Install and Import Dependencies
- MediaPipe
- OpenCV

In [1]:
# Only the first time if not yet installed
!pip3 install mediapipe opencv-python

In [24]:
import cv2
import mediapipe as mp
import urllib.request
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
import PyQt5
from PIL import Image
from IPython.display import Video
#import nb_helpers
import pandas as pd
import glob
import numpy.linalg as LA
import os
import math

import json

In [25]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
mp_face_mesh = mp.solutions.face_mesh
poselandmarks_list = mp.solutions.pose.PoseLandmark

## 1. Basic Functions

![blazepose_keypoints](https://camo.githubusercontent.com/7fbec98ddbc1dc4186852d1c29487efd7b1eb820c8b6ef34e113fcde40746be2/68747470733a2f2f6d65646961706970652e6465762f696d616765732f6d6f62696c652f706f73655f747261636b696e675f66756c6c5f626f64795f6c616e646d61726b732e706e67)

1. PoseLandmark.NOSE
2. PoseLandmark.LEFT_EYE_INNER
3. PoseLandmark.LEFT_EYE
4. PoseLandmark.LEFT_EYE_OUTER
5. PoseLandmark.RIGHT_EYE_INNER
6. PoseLandmark.RIGHT_EYE
7. PoseLandmark.RIGHT_EYE_OUTER
8. PoseLandmark.LEFT_EAR
9. PoseLandmark.RIGHT_EAR
10. PoseLandmark.MOUTH_LEFT
11. PoseLandmark.MOUTH_RIGHT
12. PoseLandmark.LEFT_SHOULDER (needed)
13. PoseLandmark.RIGHT_SHOULDER (needed)
14. PoseLandmark.LEFT_ELBOW (needed)
15. PoseLandmark.RIGHT_ELBOW (needed)
16. PoseLandmark.LEFT_WRIST (needed)
17. PoseLandmark.RIGHT_WRIST (needed)
18. PoseLandmark.LEFT_PINKY (needed)
19. PoseLandmark.RIGHT_PINKY (needed)
20. PoseLandmark.LEFT_INDEX 
21. PoseLandmark.RIGHT_INDEX
22. PoseLandmark.LEFT_THUMB (needed)
23. PoseLandmark.RIGHT_THUMB (needed)
24. PoseLandmark.LEFT_HIP (needed)
25. PoseLandmark.RIGHT_HIP (needed)
26. PoseLandmark.LEFT_KNEE (needed)
27. PoseLandmark.RIGHT_KNEE (needed)
28. PoseLandmark.LEFT_ANKLE (needed)
29. PoseLandmark.RIGHT_ANKLE (needed)
30. PoseLandmark.LEFT_HEEL
31. PoseLandmark.RIGHT_HEEL
32. PoseLandmark.LEFT_FOOT_INDEX
33. PoseLandmark.RIGHT_FOOT_INDEX

In [74]:
# The json file generated by running BlazePose
'''curl'''
#file_name = '018_barbell_curls4'

'''raise'''
#file_name = '020_barbell_front_raise'

'''fly'''
file_name = '026_dumbbell_flys'
file_path = 'output_json/' + file_name + '.json'

**1.**`read_blazepose_json_file(...)`

Extract only the needed keypoints (as shown above) for later use from the BlazePose model generated json file.

- *Parameters*: the string of the file path
- *Returns*: a dataframe with the `x`, `y` and `z` of each key point as a separate column.

In [4]:
# Process BlazePose generated JSON files
def read_blazepose_json_file(file_path):

    # 1. Load the json file generated by BlazePose, parsed_data is a list of dictionaries
    # with each dictionary as 
    with open(file_path) as data:
        all_data = data.read()

    parsed_data = json.loads(all_data)

    # 2. Create a pandas dataframe with 34 columns (frame_id and 33 key points)
    # Each row representing a frame of the video.
    list_by_keypoints = []
    i = 0
    
    while i < (len(parsed_data)/33):
        frame_dict = []
        
        for j in range(0, 33):
            frame_dict.append(parsed_data[i * 33 + j])
            
        list_by_keypoints.append(frame_dict)
        i += 1
         

    # 3. Add header row to dataframe (as in the below format)
    '''
          |frame_id|LShoulder|RShoulder|LElbow|RElbow|LWrist|RWrist|LHip|RHip|LKnee|RKnee|LAnkle|RAnkle|LPinky|RPinky|LThumb|RThumb
    ------------------------------------------------------------------------------------------
    row 0 |  0     | 0.269.  | 0.428   | ...
    ------------------------------------------------------------------------------------------
    row 1 |  1     | ...
    ...
    '''
    
    all_pose_coords = pd.DataFrame(list_by_keypoints, columns = ['Nose', 'LEye_In', 'LEye', 'LEye_Out',
                                                              'REye_In', 'REye', 'REye_Out', 'LEar', 'REar',
                                                              'LMouth', 'RMouth','LShoulder', 'RShoulder', 
                                                              'LElbow', 'RElbow', 'LWrist', 'RWrist',
                                                              'LPinky', 'RPinky', 'LIndex', 'RIndex',
                                                              'LThumb', 'RThumb', 'LHip', 'RHip', 
                                                              'LKnee', 'RKnee', 'LAnkle', 'RAnkle',
                                                              'LHeel', 'RHeel', 'LFoot_Idx', 'RFoot_Idx'])
    
    # 4. Drop irrelevant columns
    # Drop 'Nose', 'LEye_In', 'LEye', 'LEye_Out', 'REye_In', 'REye', 'REye_Out', 'LEar', 'REar', 'LMouth', 'RMouth', 'LPinky', 'RPinky', 'LIndex', 'RIndex', 'LThumb', 'RThumb', 'LHeel', 'RHeel', 'LFoot_Idx', 'RFoot_Idx'
    pose_coords = all_pose_coords.drop(columns = ['Nose', 'LEye_In', 'LEye', 'LEye_Out', 'REye_In', 
                                              'REye', 'REye_Out', 'LEar', 'REar', 
                                              'LMouth', 'RMouth', 'LIndex', 'RIndex', 
                                              'LHeel', 'RHeel', 'LFoot_Idx', 'RFoot_Idx'])
    
    # Add a column for the index of frame (optional)
    # pose_coords.insert(0, 'frame_id', range(0, len(pose_coords))) # 0 - location
    
    
    # 5. Extract the x, y, z of each keypoint
    lshoulder_x, lshoulder_y, lshoulder_z, rshoulder_x, rshoulder_y, rshoulder_z = ([] for i in range(6))
    lelbow_x, lelbow_y, lelbow_z, relbow_x, relbow_y, relbow_z = ([] for i in range(6))
    lwrist_x, lwrist_y, lwrist_z, rwrist_x, rwrist_y, rwrist_z = ([] for i in range(6))
    lhip_x, lhip_y, lhip_z, rhip_x, rhip_y, rhip_z = ([] for i in range(6))
    lknee_x, lknee_y, lknee_z, rknee_x, rknee_y, rknee_z = ([] for i in range(6))
    lankle_x, lankle_y, lankle_z, rankle_x, rankle_y, rankle_z = ([] for i in range(6))
    lpinky_x, lpinky_y, lpinky_z, rpinky_x, rpinky_y, rpinky_z = ([] for i in range(6))
    lthumb_x, lthumb_y, lthumb_z, rthumb_x, rthumb_y, rthumb_z = ([] for i in range(6))

    # Add values of each frame to each list
    # Shoulders
    for each_frame in pose_coords['LShoulder']:
        lshoulder_x.append(each_frame.get("X"))
        lshoulder_y.append(each_frame.get("Y"))
        lshoulder_z.append(each_frame.get("Z"))
    
    for each_frame in pose_coords['RShoulder']:    
        rshoulder_x.append(each_frame.get("X"))
        rshoulder_y.append(each_frame.get("Y"))
        rshoulder_z.append(each_frame.get("Z"))
        
    # Elbows
    for each_frame in pose_coords['LElbow']: 
        lelbow_x.append(each_frame.get("X"))
        lelbow_y.append(each_frame.get("Y"))
        lelbow_z.append(each_frame.get("Z"))
    
    for each_frame in pose_coords['RElbow']: 
        relbow_x.append(each_frame.get("X"))
        relbow_y.append(each_frame.get("Y"))
        relbow_z.append(each_frame.get("Z"))

    # Wrists
    for each_frame in pose_coords['LWrist']: 
        lwrist_x.append(each_frame.get("X"))
        lwrist_y.append(each_frame.get("Y"))
        lwrist_z.append(each_frame.get("Z"))

    for each_frame in pose_coords['RWrist']: 
        rwrist_x.append(each_frame.get("X"))
        rwrist_y.append(each_frame.get("Y"))
        rwrist_z.append(each_frame.get("Z"))

    # Hips
    for each_frame in pose_coords['LHip']: 
        lhip_x.append(each_frame.get("X"))
        lhip_y.append(each_frame.get("Y"))
        lhip_z.append(each_frame.get("Z"))
    
    for each_frame in pose_coords['RHip']: 
        rhip_x.append(each_frame.get("X"))
        rhip_y.append(each_frame.get("Y"))
        rhip_z.append(each_frame.get("Z"))

    # Knees
    for each_frame in pose_coords['LKnee']: 
        lknee_x.append(each_frame.get("X"))
        lknee_y.append(each_frame.get("Y"))
        lknee_z.append(each_frame.get("Z"))

    for each_frame in pose_coords['RKnee']: 
        rknee_x.append(each_frame.get("X"))
        rknee_y.append(each_frame.get("Y"))
        rknee_z.append(each_frame.get("Z"))
        
    # Ankles
    for each_frame in pose_coords['LAnkle']: 
        lankle_x.append(each_frame.get("X"))
        lankle_y.append(each_frame.get("Y"))
        lankle_z.append(each_frame.get("Z"))
        
    for each_frame in pose_coords['RAnkle']:         
        rankle_x.append(each_frame.get("X"))
        rankle_y.append(each_frame.get("Y"))
        rankle_z.append(each_frame.get("Z"))
   
    # Fingers
    for each_frame in pose_coords['LPinky']: 
        lpinky_x.append(each_frame.get("X"))
        lpinky_y.append(each_frame.get("Y"))
        lpinky_z.append(each_frame.get("Z"))
        
    for each_frame in pose_coords['RPinky']:         
        rpinky_x.append(each_frame.get("X"))
        rpinky_y.append(each_frame.get("Y"))
        rpinky_z.append(each_frame.get("Z"))
        
    for each_frame in pose_coords['LThumb']: 
        lthumb_x.append(each_frame.get("X"))
        lthumb_y.append(each_frame.get("Y"))
        lthumb_z.append(each_frame.get("Z"))
        
    for each_frame in pose_coords['RThumb']:         
        rthumb_x.append(each_frame.get("X"))
        rthumb_y.append(each_frame.get("Y"))
        rthumb_z.append(each_frame.get("Z"))
    
    # Add each of x, y and z coordinates to the dataframe
    pose_coords['LShoulder_X'] = lshoulder_x
    pose_coords['LShoulder_Y'] = lshoulder_y
    pose_coords['LShoulder_Z'] = lshoulder_z
    
    pose_coords['RShoulder_X'] = rshoulder_x  
    pose_coords['RShoulder_Y'] = rshoulder_y 
    pose_coords['RShoulder_Z'] = rshoulder_z 
    
    pose_coords['LElbow_X'] = lelbow_x
    pose_coords['LElbow_Y'] = lelbow_y
    pose_coords['LElbow_Z'] = lelbow_z
    
    pose_coords['RElbow_X'] = relbow_x
    pose_coords['RElbow_Y'] = relbow_y
    pose_coords['RElbow_Z'] = relbow_z
    
    pose_coords['LWrist_X'] = lwrist_x
    pose_coords['LWrist_Y'] = lwrist_y
    pose_coords['LWrist_Z'] = lwrist_z
    
    pose_coords['RWrist_X'] = rwrist_x
    pose_coords['RWrist_Y'] = rwrist_y
    pose_coords['RWrist_Z'] = rwrist_z
    
    pose_coords['LHip_X'] = lhip_x
    pose_coords['LHip_Y'] = lhip_y
    pose_coords['LHip_Z'] = lhip_z
    
    pose_coords['RHip_X'] = rhip_x 
    pose_coords['RHip_Y'] = rhip_y  
    pose_coords['RHip_Z'] = rhip_z 
    
    pose_coords['LKnee_X'] = lknee_x
    pose_coords['LKnee_Y'] = lknee_y
    pose_coords['LKnee_Z'] = lknee_z
    
    pose_coords['RKnee_X'] = rknee_x
    pose_coords['RKnee_Y'] = rknee_y
    pose_coords['RKnee_Z'] = rknee_z
    
    pose_coords['LAnkle_X'] = lankle_x
    pose_coords['LAnkle_Y'] = lankle_y
    pose_coords['LAnkle_Z'] = lankle_z
    
    pose_coords['RAnkle_X'] = rankle_x  
    pose_coords['RAnkle_Y'] = rankle_y
    pose_coords['RAnkle_Z'] = rankle_z
    
    pose_coords['LPinky_X'] = lpinky_x
    pose_coords['LPinky_Y'] = lpinky_y
    pose_coords['LPinky_Z'] = lpinky_z
    
    pose_coords['RPinky_X'] = rpinky_x  
    pose_coords['RPinky_Y'] = rpinky_y
    pose_coords['RPinky_Z'] = rpinky_z
    
    pose_coords['LThumb_X'] = lthumb_x
    pose_coords['LThumb_Y'] = lthumb_y
    pose_coords['LThumb_Z'] = lthumb_z
    
    pose_coords['RThumb_X'] = rthumb_x
    pose_coords['RThumb_Y'] = rthumb_y
    pose_coords['RThumb_Z'] = rthumb_z
    
    # 6. Drop the original columns
    pose_coords = pose_coords.drop(columns = ['LShoulder', 'RShoulder', 'LElbow', 'RElbow', 'LWrist', 'RWrist',
                                              'LHip', 'RHip', 'LKnee', 'RKnee', 'LAnkle', 'RAnkle', 
                                              'LPinky', 'RPinky', 'LThumb', 'RThumb'])
   
    return pose_coords

In [9]:
# Validate the function read_blazepose_json_file(...)
pose_coords = read_blazepose_json_file(file_path)
pose_coords

,LShoulder_X,LShoulder_Y,LShoulder_Z,RShoulder_X,RShoulder_Y,RShoulder_Z,LElbow_X,LElbow_Y,LElbow_Z,RElbow_X,...,LPinky_Z,RPinky_X,RPinky_Y,RPinky_Z,LThumb_X,LThumb_Y,LThumb_Z,RThumb_X,RThumb_Y,RThumb_Z
0,0.638988,0.384911,-0.916854,0.390948,0.344151,-0.732548,0.568302,0.515667,-1.007053,0.321387,...,-1.181006,0.567111,0.446655,-1.184546,0.448794,0.615873,-1.098744,0.564744,0.428477,-1.099106
1,0.638261,0.384655,-0.870215,0.390954,0.344821,-0.714495,0.572682,0.507140,-0.962485,0.322169,...,-1.163461,0.549901,0.447786,-1.197144,0.443475,0.615843,-1.077959,0.552915,0.434133,-1.102505
2,0.637887,0.383904,-0.857777,0.390925,0.344673,-0.713183,0.576300,0.502860,-0.944627,0.322592,...,-1.145445,0.544111,0.449179,-1.195913,0.439097,0.615295,-1.061492,0.545962,0.435983,-1.098735
3,0.637917,0.383841,-0.868267,0.391086,0.344506,-0.716555,0.577248,0.502305,-0.960092,0.323512,...,-1.146652,0.542608,0.449140,-1.258281,0.436973,0.615797,-1.065701,0.544015,0.435875,-1.157523
4,0.636813,0.383423,-0.827287,0.391092,0.344476,-0.688251,0.578284,0.500696,-0.899492,0.323513,...,-1.083573,0.539333,0.451139,-1.201425,0.436121,0.615757,-1.004659,0.543942,0.437553,-1.098584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,0.598932,0.380745,-0.316155,0.391035,0.349896,0.280175,0.513533,0.500125,-0.328292,0.288099,...,-0.098754,0.476623,0.408549,-0.624874,0.287451,0.574787,-0.071821,0.469395,0.394353,-0.555628
259,0.599703,0.379170,-0.341083,0.390939,0.348664,0.244955,0.514846,0.500267,-0.345897,0.289074,...,-0.129955,0.493657,0.410815,-0.598937,0.287675,0.580630,-0.101701,0.484112,0.402036,-0.530710
260,0.600277,0.378748,-0.369979,0.391170,0.347779,0.215442,0.515388,0.500375,-0.378510,0.291041,...,-0.173630,0.509905,0.415589,-0.607826,0.291744,0.583912,-0.142396,0.488014,0.407051,-0.541377
261,0.600271,0.378184,-0.389007,0.391116,0.347309,0.213117,0.515221,0.500459,-0.390157,0.291671,...,-0.176524,0.500752,0.421039,-0.586949,0.298374,0.587051,-0.147127,0.484806,0.411954,-0.521918


**2.**`three_dim_vectors_angle(...)`

Calculate the angle between two 3-dimensional vectors.

- *Parameters*: two 3-dimensional vectors' head and tail's `x`, `y`, `z` coordinates
- *Returns*: the angle (in degree)

In [5]:
# 4/14/2023

# In some cases, the two vectors might share the same tail
# Reference: https://swdevnotes.com/python/2021/math-3d-vectors-dot-product/
# Latest reference: https://stackoverflow.com/questions/2827393/angles-between-two-n-dimensional-vectors-in-python

""" Returns the unit vector of the vector.  """
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

""" Returns the angle in radians between vectors 'v1' and 'v2' """
def three_dim_vectors_angle(vec1_head_x, vec1_head_y, vec1_head_z, 
                            vec1_tail_x, vec1_tail_y, vec1_tail_z,
                            vec2_head_x, vec2_head_y, vec2_head_z,
                            vec2_tail_x, vec2_tail_y, vec2_tail_z):
    # Construct vectors
    # Tails of both vectors are the joint point (e.g. elbow)
    vector_1 = [vec1_head_x - vec1_tail_x, vec1_head_y - vec1_tail_y, vec1_head_z - vec1_tail_z]
    vector_2 = [vec2_head_x - vec2_tail_x, vec2_head_y - vec2_tail_y, vec2_head_z - vec2_tail_z]
    
    # Normalize vectors
    v1_u = vector_1 / np.linalg.norm(vector_1)
    v2_u = vector_2 / np.linalg.norm(vector_2)
    
    # Calculate angle
    angle_radian = np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
    
    """Alternatively
    dot_product = np.dot(unit_vec1, unit_vec2)
    angle_radian = np.arccos(dot_product) # Angle in radian
    """
    
    # Convert angle in radian to angle in degree
    angle_degree = angle_radian * 180 / math.pi
    
    return round(angle_degree, 3) # round 3 decimals

In [12]:
# Validate the function three_dim_vectors_angle(...)
# Should return 66.056
print(three_dim_vectors_angle(-3, 4, 3, 
                              0, 0, 0, 
                              -1, 5, -3, 
                              0, 0, 0))

66.056


**3.**`get_bend_extend(...)`

Get the turning point of each key joint when it starts to extend or bend. e.g. the left arm extends from frame number 2 to 75, and bends from frame number 75 to 120.

- *Parameters:* (1) joint_angles: `[angle1, angle2, ... anglen]`, the angle of a joint at each time frame; (2) `threshold`: the threshold angle that a change greater than which is considered a movement for this gesture (e.g. elbow angle changes greater than 60 degrees to be considered rowing).

- *Return:* The starting and ending frame number of **extend** and **bend** as a dictionary, e.g. `{'bend':[[0, 25], [75, 100]], 'extend':[[25, 75], [100, 120]]}`

In [6]:
# 5/28/2023

def get_bend_extend(joint_angles, threshold):
    
    joint_turning_points = {}

    # Check if the given movement ever occurred (if the max and min angle difference exceeds the threshold degree)
    if (max(joint_angles) - min(joint_angles)) > threshold:
        joint_bend = []
        joint_extend = []
        
        # Go from frame to frame, find the next point with the greatest difference
        # If difference <= threshold --> staying still
        # Else
        #     If difference is positive --> angle degrees going down --> bending
        #     If difference is negative --> angle degrees going up --> extending
        curr_frame = 0
        curr_angle = joint_angles[curr_frame]
        max_diff = 0
        
        # Used to save the starting and ending frame number
        interval = []
        interval.append(curr_frame)
        
        for i in range(1, len(joint_angles) - 2):
            next_angle = joint_angles[i]
            diff = abs(curr_angle - next_angle)
            
            if diff > max_diff and i < len(joint_angles) - 3:
                max_diff = diff

            else:
                # If the turning point is within threshold degree' of change with the starting point
                # Or if the degree keeps decreasing, but not reaching the local min
                if (diff < threshold) or ((curr_angle - next_angle < 0) and (next_angle < joint_angles[i+2])):
                    continue
                elif (diff < threshold) or ((curr_angle - next_angle > 0) and (next_angle > joint_angles[i+2])):
                    continue
                else:
                    interval.append(i - 1)
                    # Check if it is bending or extending
                    if curr_angle - next_angle > 0: # bending
                        joint_bend.append(interval)
                    else: # extending
                        joint_extend.append(interval)

                    curr_angle = joint_angles[i - 1]
                    interval = []
                    interval.append(i - 1)
                    max_diff = 0
        
        # Another condition is that it keeps growing/decreasing
        # but reaching the end
        if abs(curr_angle - joint_angles[-1]) > threshold:
            interval = []
            interval.append(curr_frame)
            interval.append(len(joint_angles)-1)
            if curr_angle - joint_angles[-1] > 0: # bending
                joint_bend.append(interval)
            else:  # extending
                joint_extend.append(interval)
        
        joint_turning_points['bend'] = joint_bend
        joint_turning_points['extend'] = joint_extend

    return joint_turning_points

**4.**`get_coords_lists(...)`

Extract the `x`, `y` and `z` coordinates by each key joint. e.g. `lshoulder_coords` contains `[[lshoulder_frame1_x, lshoulder_frame1_y, lshoulder_frame1_z], ... [lshoulder_framen_x, lshoulder_framen_y, lshoulder_framen_z]]`.

- *Parameters:* the dataframe of all key joints' coordinates at each time frame.

- *Return:* the list of coordinates of each key joint

In [7]:
# 5/27/2023
def get_coords_lists(pose_coords):
    
    # Save the x, y, z of each frame to a list
    lshoulder_coords = []
    rshoulder_coords = []
    lelbow_coords = [] 
    relbow_coords = []
    lwrist_coords = [] 
    rwrist_coords = []
    lhip_coords = [] 
    rhip_coords = []
    lknee_coords = []
    rknee_coords = []
    lankle_coords = []
    rankle_coords = []
    lpinky_coords = []
    rpinky_coords = []
    lthumb_coords = []
    rthumb_coords = []
    
    # Iterate through each video frame, and get x, y, z respectively.
    for index, row in pose_coords.iterrows():
        # 1. Shoulders
        lshoulder_x = row['LShoulder_X']
        lshoulder_y = row['LShoulder_Y']
        lshoulder_z = row['LShoulder_Z']
        lshoulder_coords.append([lshoulder_x, lshoulder_y, lshoulder_z])
        
        rshoulder_x = row['RShoulder_X']
        rshoulder_y = row['RShoulder_Y']
        rshoulder_z = row['RShoulder_Z']
        rshoulder_coords.append([rshoulder_x, rshoulder_y, rshoulder_z])
        
        # 2. Elbows
        lelbow_x = row['LElbow_X']
        lelbow_y = row['LElbow_Y']
        lelbow_z = row['LElbow_Z']
        lelbow_coords.append([lelbow_x, lelbow_y, lelbow_z])
        
        relbow_x = row['RElbow_X']
        relbow_y = row['RElbow_Y']
        relbow_z = row['RElbow_Z']
        relbow_coords.append([relbow_x, relbow_y, relbow_z])
        
        # 3. Wrists
        lwrist_x = row['LWrist_X']
        lwrist_y = row['LWrist_Y']
        lwrist_z = row['LWrist_Z']
        lwrist_coords.append([lwrist_x, lwrist_y, lwrist_z])

        rwrist_x = row['RWrist_X']
        rwrist_y = row['RWrist_Y']
        rwrist_z = row['RWrist_Z']
        rwrist_coords.append([rwrist_x, rwrist_y, rwrist_z])

        # 4. Hips
        lhip_x = row['LHip_X']
        lhip_y = row['LHip_Y']
        lhip_z = row['LHip_Z']
        lhip_coords.append([lhip_x, lhip_y, lhip_z])
        
        rhip_x = row['RHip_X']
        rhip_y = row['RHip_Y']
        rhip_z = row['RHip_Z']
        rhip_coords.append([rhip_x, rhip_y, rhip_z])
        
        # 5. Knees
        lknee_x = row['LKnee_X']
        lknee_y = row['LKnee_Y']
        lknee_z = row['LKnee_Z']
        lknee_coords.append([lknee_x, lknee_y, lknee_z])
        
        rknee_x = row['RKnee_X']
        rknee_y = row['RKnee_Y']
        rknee_z = row['RKnee_Z']
        rknee_coords.append([rknee_x, rknee_y, rknee_z])
        
        # 6. Ankles
        lankle_x = row['LAnkle_X']
        lankle_y = row['LAnkle_Y']
        lankle_z = row['LAnkle_Z']
        lankle_coords.append([lankle_x, lankle_y, lankle_z])
        
        rankle_x = row['RAnkle_X']
        rankle_y = row['RAnkle_Y']
        rankle_z = row['RAnkle_Z']
        rankle_coords.append([rankle_x, rankle_y, rankle_z])
        
        # 7. Pinkies
        lpinky_x = row['LPinky_X']
        lpinky_y = row['LPinky_Y']
        lpinky_z = row['LPinky_Z']
        lpinky_coords.append([lpinky_x, lpinky_y, lpinky_z])

        rpinky_x = row['RPinky_X']
        rpinky_y = row['RPinky_Y']
        rpinky_z = row['RPinky_Z']
        rpinky_coords.append([rpinky_x, rpinky_y, rpinky_z])

        # 8. Thumbs
        lthumb_x = row['LThumb_X']
        lthumb_y = row['LThumb_Y']
        lthumb_z = row['LThumb_Z']
        lthumb_coords.append([lthumb_x, lthumb_y, lthumb_z])

        rthumb_x = row['RThumb_X']
        rthumb_y = row['RThumb_Y']
        rthumb_z = row['RThumb_Z']
        rthumb_coords.append([rthumb_x, rthumb_y, rthumb_z])

    '''
    for index in range(len(lshoulder_x)):
        lshouder = [[lshoulder_x[index]], lshoulder_y[index], lshoulder_z[index]]
        lshoulder_coords.append(lshoulder)
        
        rshouder = [[rshoulder_x[index]], rshoulder_y[index], rshoulder_z[index]]
        rshoulder_coords.append(rshoulder)
    '''

    # Return the landmarks of each key joint in the following format
    # lshoulder_coords_list = [[frame_1_x, frame_1_y, frame_1_z],
    #                          [frame_2_x, frame_2_y, frame_2_z],
    #                          ... [frame_n_x, frame_n_y, frame_n_z]]
    return lshoulder_coords, rshoulder_coords, lelbow_coords, relbow_coords,lwrist_coords, rwrist_coords, lhip_coords, rhip_coords, lknee_coords, rknee_coords, lankle_coords, rankle_coords, lpinky_coords, rpinky_coords, lthumb_coords, rthumb_coords

In [15]:
# Validate the function get_coords_lists(...)
lshoulder_coords, rshoulder_coords, lelbow_coords, relbow_coords, \
lwrist_coords, rwrist_coords, lhip_coords, rhip_coords, \
lknee_coords, rknee_coords, lankle_coords, rankle_coords, \
lpinky_coords, rpinky_coords, lthumb_coords, rthumb_coords = get_coords_lists(pose_coords)
        
print(lshoulder_coords[:5])

[[0.63898766040802, 0.3849110007286072, -0.9168539047241211], [0.6382612586021423, 0.3846547305583954, -0.8702152371406555], [0.637886643409729, 0.3839041590690613, -0.8577772378921509], [0.6379168033599854, 0.38384097814559937, -0.8682670593261719], [0.6368130445480347, 0.38342341780662537, -0.8272868394851685]]


**5.**`get_single_joint_angles(...)`

- *Parameters:* the coordinates of heads and tails of two vectors.

- *Return:* angle list of that joint.

In [12]:
# 5/28/2023
# Newer version, single joint
def get_single_joint_angles(head1_coords, tail1_coords, head2_coords, tail2_coords):
    
    joint_angles = []
    
    head1_x = []
    head1_y = []
    head1_z = []
    
    tail1_x = []
    tail1_y = []
    tail1_z = []
    
    head2_x = []
    head2_y = []
    head2_z = []

    tail2_x = []
    tail2_y = []
    tail2_z = []
    
    # Iterate through each video frame
    for index in range(len(head1_coords)):
        
        # Save the angles to lists
        # vector1_head (x, y, z), vector1_tail (x, y, z), vector2_head (x, y, z), vector2_tail (x, y, z)
        joint_angles.append(three_dim_vectors_angle(head1_coords[index][0], head1_coords[index][1], head1_coords[index][2],
                                                    tail1_coords[index][0], tail1_coords[index][1], tail1_coords[index][2],
                                                    head2_coords[index][0], head2_coords[index][1], head2_coords[index][2],
                                                    tail2_coords[index][0], tail2_coords[index][1], tail2_coords[index][2]))

    return joint_angles

## 2. Definition of gestures
### ===== Upperbody Movements =====

<div>
    <h3>1. Curl</h3>
    <img src="https://ts1.cn.mm.bing.net/th/id/R-C.9fcbabee731a5602297aa650c290a9ed?rik=gctLcMyPa5PBBQ&pid=ImgRaw&r=0" width="200"/>
    <img src="https://planforfit.com/wp-content/uploads/2016/09/standing-barbell-curl.gif" width="300"/>
</div>

`ifCurl()`

- *Parameters:* the angles of left and right elbows and shoulders at each time frame.
- *Returns:* `ifcurl` (boolean), the turning points of left and right arms (showing the start and ending frame number of extend and bending of arms) (empty if not curl).

*Base Pose:*
- standing
- sitting

*Angles:*

- `elbow`< -- `shoulder` --> `hip` angle remains approximately the same (**upper arms fixed**).

- `wrist` <-- `elbow` --> `shoulder` angle switched with a difference greater than 60 degrees (lower arms bend and extend).

*Device:* barbell, dumbbells

*Palms:* facing either inward (when lower arms lifted) or outward.

In [62]:
# 6/2/2023

# >>> To be explored: single-handed situation (currently just considering
# if curls ever occur)

# How to check if the values of a dictionary exist?
# https://www.geeksforgeeks.org/python-test-for-empty-dictionary-value-list/

def ifCurl(lelbow_angles, relbow_angles, lshoulder_angles, rshoulder_angles): 
        
    ifcurl = True
    lcurl_movement = {}
    rcurl_movement = {}
    
    # 1. Check if the elbows' angles changed greater than 100 degrees
    lelbow_turning_points = get_bend_extend(lelbow_angles, 100)
    relbow_turning_points = get_bend_extend(relbow_angles, 100)

    # 2. If neither of the elbows bend or extend, consider not curls
    # i.e. no bend or extend recorded
    if not any(lelbow_turning_points.values()) or not any(relbow_turning_points.values()):
        ifcurl = False
        return ifcurl, lcurl_movement, rcurl_movement
    
    # 3. Check if the shoulders' angles remain fixed
    # So that the differences of angles is NO greater than 100 degrees
    lshoulder_turning_points = get_bend_extend(lshoulder_angles, 100)
    rshoulder_turning_points = get_bend_extend(rshoulder_angles, 100)
    
    # 4. If either of the shoulders bend or extend, consider not curls
    if not any(lshoulder_turning_points.values()) == False or not any(rshoulder_turning_points.values()) == False:
        ifcurl = False
        return ifcurl, lcurl_movement, rcurl_movement
    
    # 5. In the case there are curl movements
    lcurl_movement = lelbow_turning_points
    rcurl_movement = relbow_turning_points
    
    return ifcurl, lcurl_movement, rcurl_movement

<div>
    <h3>2. Row (Weight lifting)</h3>
    <h4>Back-bent Row (with barbell)</h4>
    <img src="https://hips.hearstapps.com/hmg-prod/images/workouts/2016/03/barbellrow-1457038583.gif" width="350"/>   
    <h4>Upright Row (with dumbbells)</h4>
    <img src="https://experiencelife.lifetime.life/wp-content/uploads/2021/09/bid-upright-row.jpg" width="400"/> 
    <h4>Row with cable/rowing machine</h4>
    <img src="https://fitnessvolt.com/wp-content/uploads/2020/10/cable-standing-row-back-exercise-750x393.jpg" width="400"/> 
</div>

`ifRow()`

- *Parameters:* the angles of left and right elbows and shoulders at each time frame.
- *Returns:* `ifrow` (boolean), the turning points of left and right arms (showing the start and ending frame number of extend and bending of arms) (empty if not curl).

*Base Pose:*

- standing (knees either bent or straight)
- sitting

*Angles:*

- `wrist` <-- `elbow` --> `shoulder` angle switches between 90 to 180 degrees.

- `elbow` <-- `shoulder` --> `hip` angle switched between 0 to 30 degrees.

*Device:* barbell, dumbbells, rowing machine

In [68]:
# 6/2/2023

# >>> To be explored: single-handed situation (currently just considering
# if rows ever occur)

def ifRow(lelbow_angles, relbow_angles, lshoulder_angles, rshoulder_angles):
        
    ifrow = True
    lrow_movement = {}
    rrow_movement = {}
    
    # 1. Check if the elbows' angles differ by angles greater than 120 degrees
    lelbow_turning_points = get_bend_extend(lelbow_angles, 120)
    relbow_turning_points = get_bend_extend(relbow_angles, 120)

    # 2. If neither of the elbows bend or extend, consider not rows
    if not any(lelbow_turning_points.values()) or not any(relbow_turning_points.values()):
        ifrow = False
        return ifrow, lrow_movement, rrow_movement
    
    # 3. Check if the shoulders' angles also bend and extend
    # So that the difference of angles is greater than 60 degrees
    lshoulder_turning_points = get_bend_extend(lshoulder_angles, 60)
    rshoulder_turning_points = get_bend_extend(rshoulder_angles, 60)
    
    # 4. If neither of the shoulders bend or extend, consider not rows
    if not any(lshoulder_turning_points.values()) or not any(rshoulder_turning_points.values()):
        ifrow = False
        return ifrow, lrow_movement, rrow_movement
    
    # 5. In the case there are row movements
    lrow_movement = lelbow_turning_points
    rrow_movement = relbow_turning_points
    
    return ifrow, lrow_movement, rrow_movement

<div>
    <h3>3. Press</h3>
    <img src="https://thumbs.gfycat.com/FluffyCarefulBaboon-max-1mb.gif" width="300"/>
</div>

`ifPress()`

*Base Pose:*

- standing
- lying (on bench)

*Angles:*

- `wrist` <-- `elbow` --> `shoulder` angle from approximately 30 to 180 degrees, repeatedly.

- `wrist` <-- `shoulder` --> `hip` angle from approximately 30 to 180 degrees, repeatedly. The changing rate same with the abovementioned one.


*Distance:*
`wrists` and `elbows` getting closer and further, respectively.

*Device:* dumbbells, (bench)

*Other:* Palms facing front (?)

In [ ]:
def ifPress():
    pass

<h3>4. Fly</h3>
<div>
<img src="https://weighttraining.guide/wp-content/uploads/2016/11/dumbbell-fly-resized.png" align = "center" width="50%"/>
</div>

<div>
    <img src="https://thumbs.gfycat.com/FragrantClosedBettong-max-1mb.gif" align = "left" width="35%"/>
    <img src="https://thumbs.gfycat.com/EverlastingUniformHatchetfish-max-1mb.gif" align = "center" width="35%"/>
</div>

`ifFly()`

*Base Pose:*

- standing
- lying (on bench)

*Angles:*

- `wrist` <-- `elbow` --> `shoulder` angle remains approximately 180 degrees (straight arms, neither bend nor extend)

- `wrist` <-- `shoulder` --> `hip` angle remains approximately 90 degrees (doesn't change much).


*Distance:*
`wrists` and `elbows` approximate to 0 and away repeatedly.

*Device:* dumbbells, (bench)

*Palms:* Facing each other <--> facing forward

In [77]:
# 5/28/2023
def ifFly(lelbow_angles, relbow_angles, lrarms_angles):
    
    iffly = True
    lrarms_movement = {}
    
    # 1. Check if the arms remain straight
    lelbow_turning_points = get_bend_extend(lelbow_angles, 120)
    relbow_turning_points = get_bend_extend(relbow_angles, 120)
    
    # If any of the arms bent --> no
    if not any(lelbow_turning_points.values()) == False or not any(relbow_turning_points.values()) == False:
        iffly = False
        return iffly, lrarms_movement
        
    # 2. Check if the angles of two arms
    # wrist <-- shoulder of each angles changes with a difference greater than 120
    lrarms_turning_points = get_bend_extend(lrarms_angles, 120)
    
    if not any(lrarms_turning_points.values()):
        iffly = False
        return iffly, lrarms_movement

    # 3. If considered fly
    lrarms_movement = lrarms_turning_points
    
    return iffly, lrarms_movement

<h3>5. Raise</h3>
<div>
<img src="https://weighttraining.guide/wp-content/uploads/2016/10/Dumbbell-Standing-Alternate-Front-Raise-resized.png" align = "center" width="50%"/>
</div>

<div>
    <img src="https://thumbs.gfycat.com/GlaringCaringBuffalo-max-1mb.gif" align = "left" width="35%"/>
    <img src="https://post.healthline.com/wp-content/uploads/2019/11/400x400_Great_Upper_Body_Exercises_for_Women_Dumbbell_Front_Raise.gif" align = "center" width="35%"/>
</div>

`ifArmRaise()`

*Categories:* 

- front raise, side raise (not considered for now)
- double-handed, single-handed (are there? not considered for now)

*Base Pose:*

- standing, sitting

*Arms:* straight, dynamic

- `wrist` <-- `elbow` --> `shoulder` angle remains approximately 180 degrees (straight arms, neither bend nor extend)

- `wrist` <-- `shoulder` --> `hip` angle switched between 0 to 90 degrees. For the algorithm, consider a differenct greater than 60 degrees a movement.

*Legs:* straight, stable

- `hip` <-- `knee` --> `ankle` angle remains approximately 180 degrees (straight arms, neither bend nor extend)

*Device:* dumbbells, barbell

*Palms:* Facing ground when raising

*Videos:* 

- `020_barbell_front_raise`

In [71]:
# 5/27/2023
# Only consider double-handed for now
def ifArmRaise(lelbow_angles, relbow_angles, lshoulder_angles, rshoulder_angles, lknee_angles, rknee_angles):
    
    if_arm_raise = True
    lshoulder_movement = {}
    rshoulder_movement = {}
    
    # 1. Check if the arms remain straight
    lelbow_turning_points = get_bend_extend(lelbow_angles, 120)
    relbow_turning_points = get_bend_extend(relbow_angles, 120)
    
    # If any of the arms bent --> no
    # If either one of the dictionaries is not empty
    if not any(lelbow_turning_points.values()) == False or not any(relbow_turning_points.values()) == False:
        if_arm_raise = False
        return if_arm_raise, lshoulder_movement, rshoulder_movement
        
    # 2. Check if the wrist <-- shoulder --> hip angles switch between 0 and 60 (approx)
    lshoulder_turning_points = get_bend_extend(lshoulder_angles, 60)
    rshoulder_turning_points = get_bend_extend(rshoulder_angles, 60)
    
    # If any of the shoulder angle does not change much --> not moving arms --> no
    # Could be single handed
    if not any(lshoulder_turning_points.values()) or not any(rshoulder_turning_points.values()):
        if_arm_raise = False
        return if_arm_raise, lshoulder_movement, rshoulder_movement
    
    # 3. Check the palms directions when lifting up
    lshoulder_movement = lshoulder_turning_points
    rshoulder_movement = rshoulder_turning_points
    
    # Check at the ending of exntend frame, if palm facing ground
    return if_arm_raise, lshoulder_movement, rshoulder_movement

The only frames concerned are the turning points and 3 frames afterwards (except for the last frame). This requires the bending and extending movements of elbows or shoulders.

In [52]:
lshoulder_coords, rshoulder_coords, lelbow_coords, relbow_coords, \
    lwrist_coords, rwrist_coords, lhip_coords, rhip_coords, \
    lknee_coords, rknee_coords, lankle_coords, rankle_coords, \
    lpinky_coords, rpinky_coords, lthumb_coords, rthumb_coords = get_coords_lists(pose_coords)

lshoulder_angles = get_single_joint_angles(lwrist_coords, lelbow_coords, lshoulder_coords, lelbow_coords)
lshoulder_turning_points = get_bend_extend(lshoulder_angles, 60)
print(lshoulder_turning_points)

{'bend': [[19, 65], [110, 149], [187, 225]], 'extend': [[0, 19], [65, 110], [149, 187]]}


<h3>6. Deadlift</h3>
<div>
<img src="https://cdn-0.weighttraining.guide/wp-content/uploads/2016/05/Barbell-Deadlift-1.png?ezimgfmt=ng%3Awebp%2Fngcb4" align = "center" width="50%"/>
</div>

<div>
    <img src="https://hips.hearstapps.com/hmg-prod/images/workouts/2016/03/barbelldeadlift-1457038089.gif" align = "left" width="35%"/>
    <img src="https://i0.wp.com/post.healthline.com/wp-content/uploads/2019/09/400x400_What’s_the_Difference_Between_Deadlifts_and_Squats_and_Which_Is_Better_for_Building_Lower_Body_Strength_Deadlift.gif?w=1155&h=840" align = "center" width="35%"/>
</div>

`ifDeadlift()`

*Parameters:*
`lshoulder_coords, rshoulder_coords, lelbow_coords, relbow_coords,
lwrist_coords, rwrist_coords, lhip_coords, rhip_coords,
lknee_coords, rknee_coords, lankle_coords, rankle_coords,
lpinky_coords, rpinky_coords, lthumb_coords, rthumb_coords`

*Returns*

*Base Pose:*

- standing

*Arms:*
- `wrist` <-- `elbow` --> `shoulder` angle remains approximately 180 degrees (straight arms, neither bend nor extend)
- `elbow` <-- `shoulder` --> `hip` angle remains approximately 180 degrees (straight arms, neither bend nor extend)


*Legs:*

*Angles:*

- `wrist` <-- `elbow` --> `shoulder` angle remains approximately 180 degrees (straight arms, neither bend nor extend)

- `wrist` <-- `shoulder` --> `hip` angle remains approximately 90 degrees.


*Distance:*
`wrists` and `elbows` approximate to 0 and away repeatedly.

*Device:* barbell

*Other:* fist palms facing the ground when raising

In [ ]:
def ifDeadlift():
    pass

`main()`

*Parameter:* file_path

*Workflow:*
- (1) Convert the BlazePose generated JSON file to pandas dataframe;
- (2) Extract the coordinate lists of all key joints; 
- (3) Calculate angles of each key joint at each time frame;
- (4) Check for upperbody movements (arms);
- (5) [TBC] Check for lowerbody movements (legs);
- (6) [TBC] Check for devices.

* Returns:* a description of what gesture is made for how many times, e.g. `"3 times of flies"`.

In [1]:
# 5/27/2023
def main(file_path):
    
    movement_description = ""
    
    # 1. Convert the BlazePose generated JSON file to pandas dataframe
    pose_coords = read_blazepose_json_file(file_path)
    
    # 2. Extract the coordinate lists of all key joints
    lshoulder_coords, rshoulder_coords, lelbow_coords, relbow_coords, \
    lwrist_coords, rwrist_coords, lhip_coords, rhip_coords, \
    lknee_coords, rknee_coords, lankle_coords, rankle_coords, \
    lpinky_coords, rpinky_coords, lthumb_coords, rthumb_coords = get_coords_lists(pose_coords)
    
    # 3. Calculate angles of each key joint (excluding wrists, ankles, pinkies and thumbs)
    lelbow_angles = get_single_joint_angles(lwrist_coords, lelbow_coords, lshoulder_coords, lelbow_coords)
    relbow_angles = get_single_joint_angles(rwrist_coords, relbow_coords, rshoulder_coords, relbow_coords)
    lshoulder_angles = get_single_joint_angles(lelbow_coords, lshoulder_coords, lhip_coords, lshoulder_coords)
    rshoulder_angles = get_single_joint_angles(relbow_coords, rshoulder_coords, rhip_coords, rshoulder_coords)
    lhip_angles = get_single_joint_angles(lshoulder_coords, lhip_coords, lknee_coords, lhip_coords)
    rhip_angles = get_single_joint_angles(rshoulder_coords, rhip_coords, rknee_coords, rhip_coords)
    lknee_angles = get_single_joint_angles(lhip_coords, lknee_coords, lankle_coords, lknee_coords)
    rknee_angles = get_single_joint_angles(rhip_coords, rknee_coords, rankle_coords, rknee_coords)

    lrarms_angles = get_single_joint_angles(lwrist_coords, lshoulder_coords, rwrist_coords, rshoulder_coords)

    # 4. Check for upperbody movements (arms)
    ifcurl, lcurl_movement, rcurl_movement = ifCurl(lelbow_angles, relbow_angles, lshoulder_angles, rshoulder_angles)
    ifrow, lrow_movement, rrow_movement = ifRow(lelbow_angles, relbow_angles, lshoulder_angles, rshoulder_angles)
    iffly, lrarms_movement = ifFly(lelbow_angles, relbow_angles, lrarms_angles)
    if_arm_raise, lshoulder_movement, rshoulder_movement = ifArmRaise(lelbow_angles, relbow_angles, lshoulder_angles, rshoulder_angles, lknee_angles, rknee_angles)

    # 5. Check for lowerbody movements (legs)
    # TBC
    
    # 6. Check for devices
    # TBC
    
    # 7. Output description
    if ifcurl == True:
        # Why getting the max value: sometimes the pose detection model
        # might not be able to detect the movements of both sides accurately
        num_curls = max(len(lcurl_movement['bend']), len(rcurl_movement['bend']))
        movement_description += str(num_curls) + " times of curls" 
       
    elif ifrow == True:
        num_rows = max(len(lrow_movement['bend']), len(rrow_movement['bend']))
        movement_description += str(num_rows) + " times of rows" 
    
    elif if_arm_raise == True:
        # Get the number of times of raises
        # Determined by the number of bending
        num_raises = max(len(lshoulder_movement['bend']), len(rshoulder_movement['bend']))
        movement_description += str(num_raises) + " times of raises" 
        
    elif iffly == True:
        num_flies = len(lrarms_movement['extend'])
        movement_description += str(num_flies) + " times of flies"
        
    else:
        movement_description += "No gestures of curl, row, raise or fly detected."

    return movement_description

In [ ]:
main(file_path)